In [1]:
from sklearn.model_selection import train_test_split
import unittest
import numpy as np
from keras_performer import performer as tfr
import nltk

from os import listdir
from os.path import isfile, isdir, join
from nltk.tokenize import wordpunct_tokenize

from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.optimizers import Adam
from keras.utils import to_categorical
import tensorflow as tf
import glob

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
def loadTestTrainData(filename): # e.g., 'test.npy'
    with open(filename, "rb") as f:
        a = np.load(f)
        return a

In [3]:
import csv
def readCSV(file_name):
    errlist=[]
    LBlist=[]
    with open(file_name, newline='') as csvfile:
    #讀取 CSV 檔內容，將每一列轉成一個 dictionary

        rows = csv.DictReader(csvfile)
        for row in rows: 
            RL=list(row.values())
            #print("RL[0]: ", type(RL[0]), "RL[1]: ", type(RL[1]))
            RL[1:] = list(map(int, RL[1:]))
            errs=RL[1:37]
            LB=RL[37:]
            errlist.append(errs)
            LBlist.append(LB)
    return errlist,LBlist


In [4]:
def find_first_sublist(seq, sublist, start=0):
    length = len(sublist)
    for index in range(start, len(seq)):
        if seq[index:index+length] == sublist:
            return index, index+length

In [5]:
def replace_sublist(seq, sublist, replacement):
    length = len(replacement)
    index = 0
    for start, end in iter(lambda: find_first_sublist(seq, sublist, index), None):
        seq[start:end] = replacement
        index = start + length

In [6]:
def replaceTAGS(x):
    replace_sublist(x, ['<', 'NUM_INT', '>'], ["<NUM_INT>"])
    replace_sublist(x, ['<', 'NUM_FLOAT', '>'], ["<NUM_FLOAT>"])
    replace_sublist(x, ['<', 'STRING', '>'], ["<STRING>"])
    replace_sublist(x, ['<', 'BOC', '>'], ["<BOC>"])
    replace_sublist(x, ['<', 'EOC', '>'], ["<EOC>"])
    replace_sublist(x, ['<', 'BOTM', '>'], ["<BOTM>"])
    replace_sublist(x, ['<', 'BOT', '>'], ["<BOT>"])
    replace_sublist(x, ['<', 'EOT', '>'], ["<EOT>"])
    replace_sublist(x, ['<', 'BOM', '>'], ["<BOM>"])
    replace_sublist(x, ['<', 'EOM', '>'], ["<EOM>"])
    replace_sublist(x, ['<', 'EOTM', '>'], ["<EOTM>"])
    replace_sublist(x, ['<', 'CR', '>'], ["<CR>"])
    return x

In [7]:
def parseSentence(x):	
    tokenizer = RegexpTokenizer(r"[\w']+|[].,:!?;=+-\\*/@#$%^&_(){}~|\"[]")
    tokens=[]
    state="START"
    chrs=""
    for i in range(len(x)):
        #print(ord(x[i]))
        if (ord(x[i])>255):
            inp="U"
        else:
            inp="E"

        if state=="START":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])

        elif state=="ASCII":
            if inp=="E":
                chrs += x[i]
            else:#U
                state="UNICODE"
                tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  #nltk.word_tokenize(chrs)
                chrs=""
                tokens.append(x[i])

        elif state=="UNICODE":
            if inp=="E":
                state="ASCII"
                chrs=x[i]
            else:
                state="UNICODE"
                tokens.append(x[i])
    if len(chrs)>0:
        tokens += tokenizer.tokenize(chrs) #wordpunct_tokenize(chrs)  # nltk.word_tokenize(chrs) 
    return replaceTAGS(tokens)

In [8]:
def readcode(fname):
    with open(fname,encoding = 'utf-8') as f:
        data = f.read()
        return data

In [9]:
import os
def listdir_fullpath(d):
    return [f for f in os.listdir(d)]

In [10]:
#save model for training
class TestTranslate(unittest.TestCase):
        
    def __init__(self):
        self.source_token_dict = {
            '<PAD>': 0,
            '<START>': 1,
            '<END>': 2,
            '<BOC>': 3,
            '<EOC>': 4,
            '<CR>': 5,
            '<NUM_INT>': 6,
            '<NUM_FLOAT>': 7,
            '<STRING>': 8,
        }
        
    @staticmethod
    def _build_token_dict(token_dict, token_list):
        for tokens in token_list:
            for token in tokens:
                if token not in token_dict:
                    token_dict[token] = len(token_dict)
        return token_dict
    
    def test_translate(self):
        #print("i am here: " )
        #source_file=[]
        #Set Para
        max_javaline_length = 160 #Max number of lines
        #set path
        Output_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Training\InputCSV\Split-500-reduce-binary-accuracy" #for csv
        Input_Path = "D:\Code\Project-AI-JAVA-ANNOTATION-2021\Training\InputTxt\Split-500-reduce-binary-accuracy" #for txt
        model_for_training_org_path = "Model-for-training\Default\Split-500-reduce-binary-accuracy\Max-len-1000"
        model_for_training_path = "Model-for-training-org\Default\Split-500-reduce-binary-accuracy\Max-len-1000"
        Trained_model_Path = "Trained_models\Default\Split-500-reduce-binary-accuracy\Max-len-1000\Learning-rate-3x0\Weight-1vs1\Drop-out-0.2\Epochs-1000"
        source_max_len_name = "source_max_len"
        type_weight = 1
        line_weight = 1
        learning_rate_value = 0.0001
        batch_size_num = 64
        epochs_value = 1000
        #get all txt file in input path
        in_path = (glob.glob(Input_Path + "/**/*.txt"))
        #print("in_path: ", in_path)
        #cases = listdir_fullpath(Input_Path)
        source_max_len = 0
        target_max_len = 0
        token_num = 0
        all_sample_num = 16644 #336 #16644 #all sample number
        block_num = 16644 #336 #16644 #sample num e.g 10000 sample have 10*1000
        #Set file cutting size
        training_source_max_len = 1000
        self.sl = 0
        
        import math
        for loop in range(0, math.ceil(all_sample_num/block_num)): #new version 
        #for loop in range(0, round(336/block_num)): #old version
            print("First loop: ", loop)
            source_tokens = []
            target_errors = []
            target_LB = []
            if(all_sample_num % block_num == 0):
                dirs = block_num
            else:
                dirs = block_num if loop < all_sample_num // block_num else all_sample_num % block_num
            Input_fullpath = []
            Output_fullpath = []
            #print("dirs: ", dirs)
            for i in range(dirs):
                Input_fullpath.append(in_path[loop*block_num + i])                    
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))
                    #print("source_tokens length: ", len(source_tokens))
                #if len(source_tokens)>max_files: break
            #get csv file     
            out_path = Output_Path + "/" + "test" + str(loop)+".csv"
            Output_fullpath = glob.glob(out_path)
            
            for f in Output_fullpath:
                if isfile(f):
                    err, lb = readCSV(f)
                    target_errors.append(err)
                    target_LB.append(lb)
                #if len(source_tokens)>max_files: break
            dd = np.asarray(target_errors)
            target_errors = target_errors[0]  
            target_LB = target_LB[0]     
            
            #change source token length
            source_tokens2 = []
            target_errors2 = []
            target_LB2 = []

            THRESHOLD_FILE_LEN = training_source_max_len
            for i in range(len(source_tokens)):
                src = source_tokens[i]
                target_error = target_errors[i]
                target_LBs = target_LB[i]
                if (len(src)<=THRESHOLD_FILE_LEN):# and  len(target)<=THRESHOLD_FILE_LEN):
                    source_tokens2.append(src)
                    target_errors2.append(target_error)
                    target_LB2.append(target_LBs)
            source_tokens = source_tokens2
            target_errors = target_errors2 #list of intgers, error types
            target_LB = target_LB2
            
            # Generate dictionaries
            self._build_token_dict(self.source_token_dict, source_tokens)
            
            # Add special tokens
            encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            
            #output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens] 
            
            self.sl = max(list(map(len, encode_tokens))+[self.sl])
            source_max_len = self.sl

        #padding here
        print("source_max_len:", source_max_len)
        saveDictionary(source_max_len, Trained_model_Path + "/" + source_max_len_name)
        for loop in range(0, math.ceil(all_sample_num/block_num)): #new version
        #for loop in range(0, round(336/block_num)): #old version
            print("Second loop: ", loop)
            source_tokens = []
            target_errors = []
            target_LB = []
            if(all_sample_num % block_num == 0):
                dirs = block_num
            else:
                dirs = block_num if loop < all_sample_num // block_num else all_sample_num % block_num
            Input_fullpath = []
            Output_fullpath = []
            for i in range(dirs):
                Input_fullpath.append(in_path[loop*block_num + i])
            for f in Input_fullpath:
                if isfile(f):
                    source_tokens.append(parseSentence(readcode(f)))

            out_path = Output_Path + "/" + "test"+ str(loop)+ ".csv"
            Output_fullpath = glob.glob(out_path)
            for f in Output_fullpath:
                if isfile(f):
                    err, lb = readCSV(f)
                    target_errors.append(err)
                    target_LB.append(lb)
            dd = np.asarray(target_errors)
            target_errors = target_errors[0]  
            target_LB = target_LB[0]
            
            #change source token length
            source_tokens2 = []
            target_errors2 = []
            target_LB2 = []

            THRESHOLD_FILE_LEN = training_source_max_len
            for i in range(len(source_tokens)):
                src = source_tokens[i]
                target_error = target_errors[i]
                target_LBs = target_LB[i]
                if (len(src)<=THRESHOLD_FILE_LEN):# and  len(target)<=THRESHOLD_FILE_LEN):
                    source_tokens2.append(src)
                    target_errors2.append(target_error)
                    target_LB2.append(target_LBs)
            source_tokens = source_tokens2
            target_errors = target_errors2 #list of intgers, error types
            target_LB = target_LB2
            
            # Generate dictionaries
            self._build_token_dict(self.source_token_dict, source_tokens)

            # Add special tokens
            #encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
            encode_tokens = [tokens for tokens in source_tokens]
            #print("encode_tokens1: ", encode_tokens)
            encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
            #print("encode_tokens2: ", encode_tokens)
            encode_input = [list(map(lambda x: self.source_token_dict[x], tokens)) for tokens in encode_tokens]
            #print("encode_input1: ", encode_input)
            token_num = len(self.source_token_dict)
            #print("token num: ", token_num)
            #print(token_num)
            #print(type(token_num))
            #define save path and save dict  
            dict_name = "source_token_dict.pickle"
            #save for org
            saveDictionary(self.source_token_dict, model_for_training_org_path + "/" + dict_name)
            #save for training
            saveDictionary(self.source_token_dict, model_for_training_path + "/" + dict_name)
            #print("x.shape", np.asarray(encode_input).shape)  #x.shape (2,  9)
            
            #x=[np.array(encode_input * 1)]
            #y=[np.array(target_errors * 1),np.array(target_LB * 1)]

            #print("x.shape", np.asarray(x).shape)  #x.shape (2, 2048, 9)

            ####  Split the data set into train and test_model
            x = np.asarray(encode_input)
            y = list(zip(np.asarray(target_errors), np.asarray(target_LB)))
            print("x.shape: ", x.shape)
            ynparray = np.asarray(y)
            print("y.shape: ", ynparray.shape)
            #save x y file 
            print("=============Saving x&y file=============")
            saveTestTrainData(model_for_training_org_path + "/" + "x" + str(loop) + ".npy", x)
            saveTestTrainData(model_for_training_org_path + "/" + "y" + str(loop) + ".npy", ynparray)
            print("===========File save completed===========")
            #print("x.shape: ", x.shape)
            #print("y length: ", len(y))
            x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=42)
            
            y_test = list(zip(*y_test))
            y_test[0] = np.asarray(y_test[0])
            y_test[1] = np.asarray(y_test[1])
            y_test[1] = to_categorical(y_test[1], num_classes = max_javaline_length) 
            #y_test = list(zip(y_test[0], y_test[1])) 
            y_test[1] = np.split(y_test[1], indices_or_sections = len(target_LB[0]), axis = 1) 
            y_test[1] = [np.squeeze(elm, axis = 1) for elm in y_test[1]]           
            
            #=============================#
            x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size = 0.1, random_state=42)
            """
            print("Split x_train shape: ", (x_train).shape)
            print("Split x_validation shape: ", (x_validation).shape)
            print("Split x_train length: ", len(x_train))
            print("Split x_validation lenght: ", len(x_validation))
            """
            buffer_train_num = len(x_train)
            buffer_val_train_num = len(x_validation)
            
            y_train = list(zip(*y_train))  
            y_train[0] = np.asarray(y_train[0])
            y_train[1] = np.asarray(y_train[1])

            y_train[1] = to_categorical(y_train[1], num_classes=max_javaline_length) #make error type vector to binary matrix
            #y_train = list(zip(y_train[0], y_train[1]))

            #print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX:" , y_train[0].shape)
            #print("y_train[1].shape", y_train[1].shape)
            #print("???????????: y_train.length ", len(y_train))
            #print("???????????: y_train.[1] type ", type(y_train[1]))
            y_train[1] = np.split(y_train[1], indices_or_sections=len(target_LB[0]), axis=1)
            y_train[1] = [np.squeeze(elm, axis = 1) for elm in y_train[1]]
            #print("after change->len(y_train[1].shape)", len(y_train[1]) )

            y_validation = list(zip(*y_validation))
            y_validation[0] = np.asarray(y_validation[0])
            y_validation[1] = np.asarray(y_validation[1])
            y_validation[1] = to_categorical(y_validation[1], num_classes = max_javaline_length) 
            #y_test = list(zip(y_test[0], y_test[1])) 
            y_validation[1] = np.split(y_validation[1], indices_or_sections=len(target_LB[0]), axis=1) 
            y_validation[1] = [np.squeeze(elm, axis = 1) for elm in y_validation[1]]           
            
        #''' <----save switch, if you need to create npy model for traing open this
            #save org model for training 
            saveTestTrainData(model_for_training_org_path + "/" + "x_train_" + str(loop) + ".npy", x_train)
            saveTestTrainData(model_for_training_org_path + "/" + "y_train[0]_" + str(loop) + ".npy", y_train[0])
            saveTestTrainData(model_for_training_org_path + "/" + "y_train[1]_" + str(loop) + ".npy", y_train[1])
            
            saveTestTrainData(model_for_training_org_path + "/" + "x_test_" + str(loop) + ".npy", x_test)
            saveTestTrainData(model_for_training_org_path + "/" + "y_test[0]_" + str(loop) + ".npy", y_test[0])
            saveTestTrainData(model_for_training_org_path + "/" + "y_test[1]_" + str(loop) + ".npy", y_test[1])
            
            #=========================================
            saveTestTrainData(model_for_training_org_path + "/" + "x_validation_" + str(loop) + ".npy", x_validation)
            saveTestTrainData(model_for_training_org_path + "/" + "y_validation[0]_" + str(loop) + ".npy", y_validation[0])
            saveTestTrainData(model_for_training_org_path + "/" + "y_validation[1]_" + str(loop) + ".npy", y_validation[1])
            
            #transform x_train
            
            print("org x_train shape: ", (x_train).shape)
            print("org x_test shape: ", (x_test).shape)
            
            
            #save split model for training
            saveTestTrainData(model_for_training_path + "/" + "x_train_" + str(loop) + ".npy", x_train)
            saveTestTrainData(model_for_training_path + "/" + "y_train[0]_" + str(loop) + ".npy", y_train[0])
            saveTestTrainData(model_for_training_path + "/" + "y_train[1]_" + str(loop) + ".npy", y_train[1])
            
            saveTestTrainData(model_for_training_path + "/" + "x_test_" + str(loop) + ".npy", x_test)
            saveTestTrainData(model_for_training_path + "/" + "y_test[0]_" + str(loop) + ".npy", y_test[0])
            saveTestTrainData(model_for_training_path + "/" + "y_test[1]_" + str(loop) + ".npy", y_test[1])
            
            #=====================================
            saveTestTrainData(model_for_training_path + "/" + "x_validation_" + str(loop) + ".npy", x_validation)
            saveTestTrainData(model_for_training_path + "/" + "y_validation[0]_" + str(loop) + ".npy", y_validation[0])
            saveTestTrainData(model_for_training_path + "/" + "y_validation[1]_" + str(loop) + ".npy", y_validation[1])
        print("Training model save successful...")    
        #'''
        
        
        #start training
        from Data_buffer import DataGeneratorTrain as DGTrain
        from Data_buffer import DataGeneratorValidation as DGValidation
        from Data_buffer import DataBuffer as db
        from random import randrange
        #Set driver path
        #source_max_len = 1200 #set max len  #default : 2889
        line_block_num = 360 #lbNum
        source_token_dict_name = "source_token_dict.pickle"
        #load source_token_dict
        source_token_dict = loadDictionary(model_for_training_path + "/" + source_token_dict_name)
        #Set model para    
        model = tfr.get_model(max_input_len=(source_max_len),
                              max_javaline_length=160,
                              errNum=36,
                              lbNum=line_block_num, #lbNum=len(target_LB[0]), #160
                              token_num=len(source_token_dict),
                              embed_dim=128, #32, try 32 or 64
                              encoder_num=6, #2 max = 6
                              head_num=4,#4
                              hidden_dim=512, #128
                              dropout_rate=0.05 #0.05
                             )
        #Set losses
        losses = {"error_feed_forward_output1": "binary_crossentropy"}
        #error type weight
        lossWeights = {"error_feed_forward_output1": type_weight}
        metrics = {"error_feed_forward_output1": "binary_accuracy"}
        #metrics = {"error_feed_forward_output1": tf.keras.metrics.Accuracy()}
        
        #error line weight
        for i in range(line_block_num):
            name = "LNout" + str(i)
            losses[name] = "categorical_crossentropy"
            lossWeights[name] = line_weight #error_feed_forward_output2[] weight # 100
            metrics[name] = tf.keras.metrics.CategoricalAccuracy()
        
        
        #set complie para
        model.compile(optimizer=Adam(learning_rate=learning_rate_value), loss=losses, loss_weights=lossWeights, metrics=metrics)
        
        
        
        #''' <-----traing switch
        #Start training
        
        print("Strat training...")    
        #set check point 
        model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = Trained_model_Path + "/" + "checkpoint_model.h5",
                                                                       monitor = "val_loss",
                                                                       mode = "min",
                                                                       save_best_only = True
                                                                      )
        
        history = model.fit(
                            x = x_train,
                            y = [y_train[0]] + y_train[1],
                            epochs = epochs_value, #100 200 500 3000
                            verbose = 2, #set visibility
                            callbacks = [model_checkpoint_callback],
                            validation_data = (x_validation, [y_validation[0]] + y_validation[1]), #-> has issue 
                            batch_size = batch_size_num
                            )
        
        
        print("Model training completed...")
        #save history
        print("Saving history...")
        saveDictionary(history.history, Trained_model_Path + "/" + "model_history")
        print("History saving completed...")
        
        #save model
        print("Saving model...")
        model.save(Trained_model_Path + "/" + "test_model1.h5")
        print("Model saving completed...")
        
        #print("history.history.keys: ", history.history.keys())\
        #'''
                
    def getsource_max_lan(self):
        return self.sl
    

In [11]:
def saveDictionary(dt, file):
        import pickle
        a_file = open(file, "wb")
        pickle.dump(dt, a_file)
        a_file.close()

In [12]:
def loadDictionary(file):
        import pickle
        a_file = open(file, "rb")
        dt = pickle.load(a_file)
        return dt

In [13]:
def saveTestTrainData(filename, data): # e.g., 'test.npy'
    with open(filename, 'wb') as f:
        np.save(f, data)

In [14]:
def loadTestTrainData(filename): # e.g., 'test.npy'    
    with open(filename, 'rb') as f:
        a = np.load(f)
        return a

In [ ]:
x=TestTranslate()
x.test_translate()

First loop:  0
source_max_len: 997
Second loop:  0
x.shape:  (16166, 997)
y.shape:  (16166, 2)
=============Saving x&y file=============
===========File save completed===========


C:\Users\W.R_Chen\anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


org x_train shape:  (13094, 997)
org x_test shape:  (1617, 997)
Training model save successful...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Start Warpping...
Strat training...
Epoch 1/1000
205/205 - 461s - loss: 541.4421 - error_feed_forward_output1_loss: 0.2326 - LNout0_loss: 4.4967 - LNout1_loss: 4.5562 - LNout2_loss: 2.8305 - LNout3_loss: 2.8156 - LNout4_loss: 2.1566 - LNout5_loss: 2.8445 - LNout6_loss: 1.4446 - LNout7_loss: 1.6354 - LNout8_loss: 1.6520 - LNout9_loss: 1.9172 - LNout10_loss: 1.2533 - LNout11_loss: 2.1798 - LNout12_loss: 1.6457 - LNout13_loss: 2.1602 - LNout14_loss: 2.8003 - LNout15_loss: 1.9437 - LNout16_loss: 2.0122 - LNout17_loss: 1.5077 - LNout18_loss: 1.5008 - LNout19_loss: 1.0715 - LNout20_loss: 1.5005 - LNout21_loss: 1.7305 - LNout22_loss: 1.2634 - LNout23_loss: 1.8128 - LNout24_loss: 1.2294 - LNout25_loss: 

C:\Users\W.R_Chen\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/1000
205/205 - 134s - loss: 40.6416 - error_feed_forward_output1_loss: 0.1856 - LNout0_loss: 3.7242 - LNout1_loss: 3.8288 - LNout2_loss: 1.9642 - LNout3_loss: 1.9613 - LNout4_loss: 0.8707 - LNout5_loss: 0.8527 - LNout6_loss: 0.5194 - LNout7_loss: 0.4761 - LNout8_loss: 0.3691 - LNout9_loss: 0.3719 - LNout10_loss: 0.2926 - LNout11_loss: 0.2890 - LNout12_loss: 0.2977 - LNout13_loss: 0.2756 - LNout14_loss: 0.2454 - LNout15_loss: 0.2405 - LNout16_loss: 0.1706 - LNout17_loss: 0.1626 - LNout18_loss: 0.1383 - LNout19_loss: 0.1488 - LNout20_loss: 0.1314 - LNout21_loss: 0.1197 - LNout22_loss: 0.1364 - LNout23_loss: 0.1274 - LNout24_loss: 0.1249 - LNout25_loss: 0.1170 - LNout26_loss: 0.1127 - LNout27_loss: 0.0985 - LNout28_loss: 0.0448 - LNout29_loss: 0.0431 - LNout30_loss: 0.0503 - LNout31_loss: 0.0415 - LNout32_loss: 0.0385 - LNout33_loss: 0.0454 - LNout34_loss: 0.0356 - LNout35_loss: 0.0380 - LNout36_loss: 0.0289 - LNout37_loss: 0.0282 - LNout38_loss: 0.0201 - LNout39_loss: 0.0215 - LN

...

Epoch 25/1000
205/205 - 125s - loss: 29.9543 - error_feed_forward_output1_loss: 0.1437 - LNout0_loss: 3.2509 - LNout1_loss: 3.3543 - LNout2_loss: 1.7040 - LNout3_loss: 1.6952 - LNout4_loss: 0.6540 - LNout5_loss: 0.6768 - LNout6_loss: 0.3532 - LNout7_loss: 0.3372 - LNout8_loss: 0.2639 - LNout9_loss: 0.2642 - LNout10_loss: 0.1958 - LNout11_loss: 0.1940 - LNout12_loss: 0.1789 - LNout13_loss: 0.1785 - LNout14_loss: 0.1465 - LNout15_loss: 0.1501 - LNout16_loss: 0.1042 - LNout17_loss: 0.1022 - LNout18_loss: 0.0779 - LNout19_loss: 0.0782 - LNout20_loss: 0.0766 - LNout21_loss: 0.0760 - LNout22_loss: 0.0703 - LNout23_loss: 0.0714 - LNout24_loss: 0.0671 - LNout25_loss: 0.0669 - LNout26_loss: 0.0535 - LNout27_loss: 0.0537 - LNout28_loss: 0.0210 - LNout29_loss: 0.0212 - LNout30_loss: 0.0183 - LNout31_loss: 0.0174 - LNout32_loss: 0.0144 - LNout33_loss: 0.0148 - LNout34_loss: 0.0132 - LNout35_loss: 0.0125 - LNout36_loss: 0.0108 - LNout37_loss: 0.0116 - LNout38_loss: 0.0097 - LNout39_loss: 0.0099 - L

Epoch 26/1000
205/205 - 127s - loss: 29.3502 - error_feed_forward_output1_loss: 0.1401 - LNout0_loss: 3.2107 - LNout1_loss: 3.3174 - LNout2_loss: 1.6794 - LNout3_loss: 1.6645 - LNout4_loss: 0.6371 - LNout5_loss: 0.6602 - LNout6_loss: 0.3476 - LNout7_loss: 0.3295 - LNout8_loss: 0.2592 - LNout9_loss: 0.2564 - LNout10_loss: 0.1890 - LNout11_loss: 0.1865 - LNout12_loss: 0.1729 - LNout13_loss: 0.1716 - LNout14_loss: 0.1413 - LNout15_loss: 0.1446 - LNout16_loss: 0.1005 - LNout17_loss: 0.0975 - LNout18_loss: 0.0743 - LNout19_loss: 0.0740 - LNout20_loss: 0.0732 - LNout21_loss: 0.0734 - LNout22_loss: 0.0666 - LNout23_loss: 0.0668 - LNout24_loss: 0.0632 - LNout25_loss: 0.0634 - LNout26_loss: 0.0509 - LNout27_loss: 0.0511 - LNout28_loss: 0.0207 - LNout29_loss: 0.0204 - LNout30_loss: 0.0176 - LNout31_loss: 0.0173 - LNout32_loss: 0.0140 - LNout33_loss: 0.0140 - LNout34_loss: 0.0125 - LNout35_loss: 0.0123 - LNout36_loss: 0.0104 - LNout37_loss: 0.0110 - LNout38_loss: 0.0091 - LNout39_loss: 0.0090 - L

Epoch 27/1000
205/205 - 125s - loss: 30.6418 - error_feed_forward_output1_loss: 0.1579 - LNout0_loss: 3.3823 - LNout1_loss: 3.4710 - LNout2_loss: 1.7597 - LNout3_loss: 1.7491 - LNout4_loss: 0.6678 - LNout5_loss: 0.6783 - LNout6_loss: 0.3546 - LNout7_loss: 0.3426 - LNout8_loss: 0.2662 - LNout9_loss: 0.2665 - LNout10_loss: 0.1950 - LNout11_loss: 0.1958 - LNout12_loss: 0.1804 - LNout13_loss: 0.1786 - LNout14_loss: 0.1474 - LNout15_loss: 0.1499 - LNout16_loss: 0.1055 - LNout17_loss: 0.1028 - LNout18_loss: 0.0800 - LNout19_loss: 0.0795 - LNout20_loss: 0.0773 - LNout21_loss: 0.0772 - LNout22_loss: 0.0714 - LNout23_loss: 0.0715 - LNout24_loss: 0.0674 - LNout25_loss: 0.0674 - LNout26_loss: 0.0551 - LNout27_loss: 0.0549 - LNout28_loss: 0.0221 - LNout29_loss: 0.0223 - LNout30_loss: 0.0190 - LNout31_loss: 0.0186 - LNout32_loss: 0.0144 - LNout33_loss: 0.0148 - LNout34_loss: 0.0130 - LNout35_loss: 0.0128 - LNout36_loss: 0.0108 - LNout37_loss: 0.0111 - LNout38_loss: 0.0096 - LNout39_loss: 0.0096 - L

Epoch 28/1000
205/205 - 122s - loss: 28.8985 - error_feed_forward_output1_loss: 0.1444 - LNout0_loss: 3.2395 - LNout1_loss: 3.3303 - LNout2_loss: 1.6629 - LNout3_loss: 1.6499 - LNout4_loss: 0.6244 - LNout5_loss: 0.6430 - LNout6_loss: 0.3395 - LNout7_loss: 0.3256 - LNout8_loss: 0.2524 - LNout9_loss: 0.2502 - LNout10_loss: 0.1825 - LNout11_loss: 0.1816 - LNout12_loss: 0.1688 - LNout13_loss: 0.1655 - LNout14_loss: 0.1356 - LNout15_loss: 0.1383 - LNout16_loss: 0.0971 - LNout17_loss: 0.0942 - LNout18_loss: 0.0741 - LNout19_loss: 0.0729 - LNout20_loss: 0.0713 - LNout21_loss: 0.0713 - LNout22_loss: 0.0659 - LNout23_loss: 0.0656 - LNout24_loss: 0.0619 - LNout25_loss: 0.0625 - LNout26_loss: 0.0503 - LNout27_loss: 0.0505 - LNout28_loss: 0.0205 - LNout29_loss: 0.0201 - LNout30_loss: 0.0178 - LNout31_loss: 0.0165 - LNout32_loss: 0.0131 - LNout33_loss: 0.0137 - LNout34_loss: 0.0124 - LNout35_loss: 0.0120 - LNout36_loss: 0.0101 - LNout37_loss: 0.0105 - LNout38_loss: 0.0090 - LNout39_loss: 0.0091 - L

Epoch 29/1000
